In [ ]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
file_path = '/content/RAINFALL_DAILY_NDCQ-2024-05-109.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

In [ ]:
data = {}
current_station = None


In [ ]:
for line in lines:
    if line.startswith('STATION'):

        parts = line.split(',')
        station_name = parts[0].split(':')[1].strip()
        data[station_name] = []
        current_station = station_name
    elif current_station and line[0].isdigit():

        parts = line.split()
        year = int(parts[0])
        month = int(parts[1])
        daily_rainfall = parts[2:]
        for day, rainfall in enumerate(daily_rainfall, start=1):
            data[current_station].append({
                'Year': year,
                'Month': month,
                'Day': day,
                'Rainfall': float(rainfall)
            })

In [ ]:
dfs = []
for station, records in data.items():
    df = pd.DataFrame(records)
    df['Station'] = station
    dfs.append(df)


final_df = pd.concat(dfs)

In [ ]:
from datetime import datetime
pivot_df = final_df.pivot_table(index=['Year', 'Month', 'Day'], columns='Station', values='Rainfall').reset_index()
def convert_date(x):
      year, month, day = int(float(x.split()[0])), int(float(x.split()[1])), int(float(x.split()[2]))
      return datetime.strptime(f"{year} {month} {day}", '%Y %m %d')


pivot_df['date_str'] = pivot_df[['Year', 'Month', 'Day']].astype(str).agg(' '.join, axis=1)

pivot_df['Date'] = pivot_df['date_str'].apply(convert_date)

pivot_df.drop(columns=['Year','Month','Day','date_str'], inplace=True)
pivot_df.set_index('Date', inplace=True)
pivot_df.head()



output_path = '/content/drive/MyDrive/IMD_DR_tests.xlsx'
pivot_df.to_excel(output_path, index=True)

Initiate further post processing
